<span style="color: orange; font-weight: bold; font-size: 26px;">🚀 Week 3 – CrewAI Framework Overview</span>

---

### 🧠 מבוא
- **CrewAI** - framework חדש לניהול סוכנים (Agents) ויצירת צוותים (Crews) של מודלים חכמים.  
- דומה במידה מסוימת ל־ **OpenAI Agents SDK**, אך עם מבנה ברור, מונחים רשמיים ופרקטיקות "מונחות".  
- כולל שלושה רכיבים מרכזיים:
  1. **CrewAI Enterprise** – פלטפורמה עסקית לניהול, ניטור ו־ Deployment של סוכנים (בתשלום).  
  2. **CrewAI Studio (UI)** – ממשק Low-Code/No-Code לבניית אינטראקציות בין סוכנים.  
  3. **CrewAI Framework (Open Source)** – הספרייה שבה אנחנו כותבים קוד בפועל (ההתמקדות בקורס).

---

### 🧩 שלושת מושגי היסוד

#### 1. **Agent**
- יחידת העבודה הקטנה ביותר – סוכן אוטונומי המבוסס על מודל שפה (LLM).  
- לכל Agent יש:
  - **Role (תפקיד)** – מה הוא עושה.
  - **Goal (מטרה)** – מה הוא מנסה להשיג.
  - **Backstory (רקע)** – ההקשר שמגדיר את זהותו או אישיותו.
  - **Memory (זיכרון)** – לשמירת הקשר בין משימות.
  - **Tools (כלים)** – פונקציות או ממשקים חיצוניים שהסוכן יכול להפעיל.

<u>Types of Memory:</u>
1. **Short-Term Memory** - שומר אינטראקציות/תכנים אחרונים בוקטורים (RAG). מאפשר לאייג’נטים “להיזכר” במה שקרה לאחרונה.
2. **Long-Term Memory**  - שומר מידע חשוב לאורך זמן בבסיס נתונים (למשל SQLite).
3. **Entity Memory**     - כמו Short-Term, אבל ממוקד ביישויות (אנשים/מקומות/מושגים) עם חיפוש דמיון וקטורי.
4. **Contextual Memory** - לא “זיכרון נוסף” אלא מונח מטריה: שילוב ה -Short/Long/Entity שמוזרם כקונטקסט לפרומפט.
5. **User Memory**    - מידע פר-משתמש. יש תמיכה רעיונית, אבל בפועל תצטרך/י לשלוף ולהזריק לפרומפט בעצמך בזמן הנכון

---

#### 2. **Task**
- משימה שמוקצית ל־ Agent מסוים.
- כוללת:
  - תיאור המשימה (`description`)
  - תוצאה צפויה (`expected_output`)
  - הסוכן המבצע (`agent`)
- Agent אחד יכול לבצע כמה Tasks שונים.

> 🧱 **Agent = מי עושה** | **Task = מה לעשות**

---

#### 3. **Crew**
- צוות של Agents ו־Tasks הפועלים יחד.  
- ניתן להריץ Crew בשני מצבי עיבוד (process modes):

| מצב | תיאור |
|------|--------|
| **Sequential** | ביצוע סדרתי של משימות לפי הסדר שהוגדר. |
| **Hierarchical** | ניהול היררכי ע"י סוכן־מנהל (LLM) שמחליט למי להקצות כל משימה. |

---

### ⚖️ השוואה ל־ OpenAI Agents SDK

| היבט | OpenAI Agents SDK | CrewAI |
|-------|--------------------|---------|
| Prompting | רק `instructions` (חופשי) | `Role` + `Goal` + `Backstory` |
| מבנה | פשוט וגמיש | מובנה ו"מונחה שיטה" |
| Debugging | שליטה מלאה ב־prompt | פחות שקוף (נוצר מאחורי הקלעים) |
| Best Practices | תלוי במשתמש | מכוון ל־prompting נכון כברירת מחדל |

> 🧠 היתרון: מעודד תכנון נכון וברור של תפקידים.  
> ⚠️ החיסרון: פחות חופש וגמישות, וקשה יותר לדבג prompt.

---

### 🗂️ קבצי קונפיגורציה (YAML)

- ניתן להגדיר Agents ו־Tasks **מחוץ לקוד**, בקובצי YAML קריאים.
- מאפשר **הפרדה בין לוגיקה לקונפיגורציה**.
  
דוגמה: agents_config.yaml
```yaml
researcher:
  role: >
    Senior Financial Researcher
  goal: >
    Research companies, news and potential
  backstory: >
    You're a seasoned financial researcher with a talent for finding the most relevant information.
  llm: openai/gpt-4o-mini
```

agent = Agent(config=self.agents_config['researcher'])

> 🧠 היתרון: ניקיון קוד, קל לשנות תפקידים או מטרות בלי לגעת בלוגיקה.<br>
> ⚠️ החיסרון: צריך להתרגל לעוד שכבת הגדרה, וזה ספציפי ל־CrewAI.

---

### 🧱 הקובץ המרכזי – `crew.py`

הקובץ שבו **הכול מתחבר ביחד**:<br>
מגדירים בו:
- את כל הסוכנים (Agents)
- את כל המשימות (Tasks)
- ואת הצוות כולו (Crew)

ההגדרות נעשות בעזרת **Decorators** , שמייצרים את המבנים ושומרים אותם ברשימות פנימיות.

<img src="attachments/crew.pyFileExample.png" width="400">

### 🎛️ ה־ Decorators המרכזיים:

| Decorator | מה הוא עוטף | מה הוא עושה בפועל |
|---|---|---|
| `@CrewBase` | מחלקת־על אחת שמייצגת את ההרכב כולו | מאתחל את התשתית של ה־Crew ושומר עבורך רשימות פנימיות של `agents` ו־`tasks` |
| `@agent` | פונקציה שמחזירה Agent | יוצר Agent ומוסיף אותו אוטומטית אל `self.agents` של המחלקה |
| `@task` | פונקציה שמחזירה Task | יוצר Task ומוסיף אותו אוטומטית אל `self.tasks` של המחלקה |
| `@crew` | פונקציה שמחזירה את ה־Crew המוכן להרצה | מרכיב את כל החלקים יחד, ובוחר מצב עיבוד (`Process.sequential` / `Process.hierarchical`) |

> 💡 המשמעות: כל פונקציה שמעוטרת ב־ `@agent`/`@task` **לא צריך** להוסיף ידנית לרשימות — זה קורה אוטומטית ע״י ה־ decorators.

---

<img src="attachments/Screenshot 2025-10-05 185922.png" width="400">

ב־CrewAI לכל Agent יש מודל שפה “מתחתיו”.<br>
את המודל מגדירים באובייקט LLM(...) (של CrewAI) שמאחוריו פועל LiteLLM.
הזיהוי נעשה במחרוזת מהצורה: <provider>/<model> 

<u>אפשר להגדיר את המודל בשתי דרכים:</u><br>
- ישירות בקוד בתוך crew.py (כמו בתמונה כאן).
- דרך YAML בתוך config/agents.yaml, ואז crew.py טוען את הקונפיגורציה.

---

<img src="attachments/Screenshot 2025-10-05 191713.png" width="400">

---

<img src="attachments/Screenshot 2025-10-05 204035.png" width="400">